In [ ]:
import os
from os.path import join as oj
import sys, time
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import pandas as pd
import data_pecarn
import data_psrc
import data
import train
import matplotlib.gridspec as grd
from data import feats_numerical, feats_categorical, meta

# sns.set(style="black")
# plt.style.use('dark_background')
outcome_def = 'iai_intervention' # output
%matplotlib inline
%load_ext autoreload
%autoreload 2

# combined

In [2]:
df_pecarn, df_psrc, common_feats, filtered_feats_pecarn, filtered_feats_psrc = data.load_it_all(dummy=True)
df = df_pecarn[common_feats].append(df_psrc[common_feats])
processed_feats = data.select_final_feats(common_feats)

computing pecarn preprocessing...
computing psrc preprocessing...


In [3]:
train_idxs = data.pecarn_train_idxs
test_idxs1 = data.pecarn_test_idxs
test_idxs2 = data.psrc_train_idxs + data.psrc_test_idxs

/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
print(len(processed_feats), processed_feats)

26 ['VomitWretch_unknown', 'AbdDistention_yes', 'MOI_Motor vehicle collision', 'InitHeartRate', 'AbdTrauma_or_SeatBeltSign_yes', 'MOI_Object struck abdomen', 'AbdTenderDegree_Severe', 'RtCostalTender', 'MOI_Bike collision/fall', 'AbdomenPain_yes', 'ThoracicTrauma_yes', 'AbdTenderDegree_Mild', 'Hypotension_yes', 'VomitWretch_yes', 'MOI_Pedestrian/bicyclist struck by moving vehicle', 'InitSysBPRange', 'MOI_Fall from an elevation', 'LtCostalTender', 'AbdTenderDegree_Moderate', 'MOI_Motorcycle/ATV/Scooter collision', 'Age', 'MOI_unknown', 'DecrBreathSound_yes', 'GCSScore_Full_yes', 'AbdDistention_unknown', 'CostalTender']


In [5]:
print(train_idxs, test_idxs1, test_idxs2)

[1, 2, 3, 4, 5] [6] [8, 9, 10, 11, 12, 13]


# predict

In [ ]:
out_dir = f'results/jun17_0'
os.makedirs(out_dir, exist_ok=True)

for balancing in ['ros', 'smote']:
    for balancing_ratio in [100, 5]: 
        for model_type in ['logistic', 'dt']: #, 'rf', 'mlp2', 'svm']): # 'rf', 'mlp2', 'svm', 'gb'
            for feature_selection in ['select_stab_lasso', 'select_lasso', 'select_rf']: # select_lasso, select_rf, None
                for feature_selection_num in tqdm([5, 10, len(processed_feats)]):
                    out_name = f'{model_type}_{feature_selection}={feature_selection_num}_{balancing}={balancing_ratio}'
                    train.train(df,
                                feat_names=processed_feats,
                                model_type=model_type, 
                                balancing=balancing,
                                outcome_def=outcome_def,
                                balancing_ratio=balancing_ratio,
                                out_name=f'{out_dir}/{out_name}.pkl',
                                feature_selection=feature_selection,
                                feature_selection_num=feature_selection_num,
                                train_idxs=train_idxs,
                                test_idxs1=test_idxs1,
                                test_idxs2=test_idxs2)

In [ ]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_random_state
from stability_selection import StabilitySelection


def _generate_dummy_classification_data(p=1000, n=1000, k=5, random_state=123321):

    rng = check_random_state(random_state)

    X = rng.normal(loc=0.0, scale=1.0, size=(n, p))
    betas = np.zeros(p)
    important_betas = np.sort(rng.choice(a=np.arange(p), size=k))
    betas[important_betas] = rng.uniform(size=k)

    probs = 1 / (1 + np.exp(-1 * np.matmul(X, betas)))
    y = (probs > 0.5).astype(int)

    return X, y, important_betas

## This is all preparation of the dummy data set
n, p, k = 500, 1000, 5

X, y, important_betas = _generate_dummy_classification_data(n=n, k=k)
base_estimator = LogisticRegression(penalty='l1', solver='liblinear')

## Here stability selection is instantiated and run
selector = StabilitySelection(base_estimator=base_estimator, lambda_name='C',
                              lambda_grid=np.logspace(-5, -1, 5), threshold=1e-999).fit(X, y)

print(selector.get_support(indices=True))

In [33]:
selector.stability_scores_.shape

(1000, 5)